In [17]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import transformers 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import datasets
torch.manual_seed(1)


In [4]:
from datasets import load_dataset
dataset = load_dataset("multi_nli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9832
    })
})


In [19]:
test_range=[i for i in range(300)]
val_range=[i+300 for i in range(700)]

In [15]:
#print(test_range)
#print(val_range)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [20]:
test_set=dataset["train"].select(test_range)
val_set=dataset["train"].select(val_range)
print(test_set)
print(val_set)

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 300
})
Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 700
})


In [22]:
test_set=test_set.remove_columns(["promptID", "pairID", "premise_binary_parse", "premise_parse",  "hypothesis_binary_parse", "hypothesis_parse", "genre"])
val_set=val_set.remove_columns(["promptID", "pairID", "premise_binary_parse", "premise_parse", "hypothesis_binary_parse", "hypothesis_parse", "genre"])

In [23]:
print(test_set)
print(val_set)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 300
})
Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 700
})


In [31]:
label_to_answers={"0":"Yes","1":"No"} #entail or not

In [26]:
def get_answer(example):
  return label_to_answers[str(example["label"])]

In [30]:
def get_reversed_answer(answer):
  if answer=="Yes":
    reversed_answer="No"
  if answer=="No":
    reversed_answer="Yes"
  return reversed_answer

In [28]:
def preprocess(example):#change label 2 to 1, means not entail
  label=example["label"]
  if label==2:
    label=1
  return {"label": label}

In [29]:
test_set=test_set.map(preprocess)
val_set=val_set.map(preprocess)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [35]:
templates={"template1":"{premise} {hypothesis}",
"template1":"Suppose {premise} Can we infer that {hypothesis}?",
"template2":"Given {premise} Is it guaranteed true that {hypothesis}?"}

In [37]:
def get_complete_example(example):#get complete example using template or not
  answer=get_answer(example)
  reversed_answer=get_reversed_answer(answer)
  return {"answer":answer,"reversed_answer":reversed_answer}

In [38]:
test_set_complete=test_set.map(get_complete_example)
val_set_complete=val_set.map(get_complete_example)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [44]:
print(test_set_complete[0])
print(val_set_complete[0])

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.', 'hypothesis': 'Product and geography are what make cream skimming work. ', 'label': 1, 'answer': 'No', 'reversed_answer': 'Yes'}
{'premise': "You're mad.", 'hypothesis': 'You are still sane.', 'label': 1, 'answer': 'No', 'reversed_answer': 'Yes'}


In [50]:
foldername= "/content/drive/My Drive/IS/"
path=foldername+"multi_nli/multi_nli_dataset/"

In [51]:
test_set_complete.save_to_disk(path+"test_data")
val_set_complete.save_to_disk(path+"fewshot_data")

Saving the dataset (0/1 shards):   0%|          | 0/300 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/700 [00:00<?, ? examples/s]